In [4]:
from flwr.server import ServerApp
from fdaopt.client import client_app

In [ ]:
run_simulation(
    server_app=server_app,
    client_app=client_app,
    num_supernodes=10,
)

INFO :      Starting Flower ServerApp, config: num_rounds=100, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.7195841272672018, {'accuracy': 0.31862745098039214, 'f1': 0.060810810810810814, 'testing_loss': 0.7195841272672018}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 5 clients (out of 10)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      fit progress: (1, 3.426889260609945, {'accuracy': 0.3161764705882353, 'f1': 0.0, 'testing_loss': 3.426889260609945}, 45.01178526878357)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 10)
INFO :      aggregate_fit: received 5 results and 0 failures
INFO :      fit progress: (2, 1.7048353541131

# Start Server

In [10]:
"""fdaopt: A Flower / HuggingFace app."""

import torch

from flwr.common import ndarrays_to_parameters
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from transformers import AutoModelForSequenceClassification

from fdaopt.training import get_weights, get_evaluate_fn

from fdaopt.parameters import load_parameters


def get_strategy_class(strategy_name):
    """
    Dynamically imports and returns the strategy class from flwr.server.strategy based on the given strategy name.

    Args:
        strategy_name (str): The name of the strategy class as a string.

    Returns:
        type: The strategy class from flwr.server.strategy
    """
    import flwr.server.strategy as flwr_strat
    
    optimizer_class = getattr(flwr_strat, strategy_name)
    return optimizer_class

# 1. Load Parameters

device = "cuda" if torch.cuda.is_available() else "cpu"

params = load_parameters()

total_rounds = params['training']['total_rounds']
fraction_fit = params['training']['clients_per_round'] / params['training']['num_clients']

model_checkpoint = params['model']['checkpoint']
num_labels = params['model']['num_labels']

ds_path = params['dataset']['path']
ds_name = params['dataset']['name']


model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=num_labels
)

strategy_name = params['server']['strategy']['name']

weights = get_weights(model)
initial_parameters = ndarrays_to_parameters(weights)

FedStrat = get_strategy_class(strategy_name)

# Define strategy
strategy = FedStrat(
    fraction_fit=fraction_fit,
    fraction_evaluate=0.0,
    initial_parameters=initial_parameters,
    evaluate_fn=get_evaluate_fn(model, device, model_checkpoint, ds_path, ds_name),
)
config = ServerConfig(num_rounds=total_rounds)

In [11]:
from flwr.server import start_server

In [ ]:
start_server(
    server_address="[::]:8081",
    config=config,
    strategy=strategy
)

	Instead, use the `flower-superlink` CLI command to start a SuperLink as shown below:

		$ flower-superlink --insecure

	To view usage and all available options, run:

		$ flower-superlink --help

	Using `start_server()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower server, config: num_rounds=100, no round_timeout
INFO :      Flower ECE: gRPC server running (100 rounds), SSL is disabled
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.6774400601200029, {'accuracy': 0.6446078431372549, 'f1': 0.7687400318979266, 'testing_loss': 0.6774400601200029}
INFO :      
INFO :      [ROUND 1]


# Start Client

In [1]:
from functools import partial

In [2]:
def f(x, y):
    return x + y

In [3]:
g = partial(f, y=3)

In [4]:
g(1)

4

In [1]:
from confluent_kafka import Consumer, KafkaError
import subprocess
import json

In [5]:
def kafka_get_test_hyper_parameters(topic='FedL', bootstrap_servers='localhost:9092', group_id='fda'):
    from confluent_kafka import Consumer, KafkaError

    # Consumer example
    c = Consumer({
        'bootstrap.servers': bootstrap_servers,
        'group.id': group_id,
        'auto.offset.reset': 'latest'
    })

    c.subscribe([topic])

    while True:
        msg = c.poll(1.0)
        if not msg:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break
        val = msg.value().decode("utf-8")

        # Parse the JSON string back into a Python data structure
        parameters = json.loads(val)

        c.close()

        return parameters

In [6]:
kafka_get_test_hyper_parameters()

{'checkpoint': 'roberta-base',
 'ds_path': 'glue',
 'ds_name': 'sst2',
 'num_labels': 2,
 'num_clients': 100,
 'clients_per_round': 10,
 'alpha': 1.0,
 'batch_size': 8,
 'local_epochs': 3,
 'max_epochs': -1,
 'total_rounds': 200,
 'fda': False,
 'theta': 0.0,
 'extras': '',
 'server_opt_name': 'Adam',
 'server_opt_lr': 0.0001,
 'client_opt_name': 'SGD',
 'client_opt_lr': 0.0001}